<a href="https://colab.research.google.com/github/muhammadanas25/Stock_trend_prediction/blob/main/Ensemble_model_(Adb_%2BRF).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Initial imports
import pandas as pd
!pip install path
from path import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
%matplotlib inline
# Initial imports
import pandas as pd
import numpy as np
from path import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import GradientBoostingClassifier
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn import metrics
from sklearn.metrics import roc_curve
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()

from sklearn.compose import make_column_selector as selector


In [3]:
# Loading data
file_path = Path("HPQ.csv")
df = pd.read_csv(file_path)
df.dropna(inplace=True)
df.tail()

,Date,Open,High,Low,Close,Volume,EF_1,EF_2,Oil FutureTrend,Gold FutureTrend,Trend,Twitter Sentiments,FutureTrend
749,23/06/2021,29.180000,29.510000,28.959999,29.200001,7914000,0.436366,0.068542,Negative,Positive,Positive,0,Negative
750,24/06/2021,29.500000,29.510000,29.110001,29.469999,6270700,0.899998,-0.101697,Negative,Negative,Negative,0,Negative
751,25/06/2021,29.400000,29.920000,29.370001,29.709999,17050100,0.618179,1.054420,Positive,Negative,Positive,0,Negative
752,28/06/2021,29.760000,30.030001,29.639999,29.809999,6570000,0.435896,0.168008,Negative,Positive,Positive,0,Negative
753,29/06/2021,29.950001,30.320000,29.790001,30.139999,7779100,0.660376,0.634386,Negative,Negative,Positive,0,Negative


In [4]:
def make_bins(df):
   label_names = ["Ef_1_v_low","Ef_1_low","Ef_1_v_med","Ef1_high", "Ef1_very_high" ]
   cut_points =[0. , 0.2, 0.4, 0.6, 0.8, 1. ]
   df["EF_1_val"] = pd.cut(df["EF_1"], cut_points, labels=label_names)
   dummies = pd.get_dummies(df["EF_1_val"],drop_first=True)
   df = pd.concat([df, dummies], axis=1)
   label_names = ["setinent_neutral","sentiment_good","sentiment_bad"]
   cut_points =[0, 0.29,0.58 ,0.87] 
   df["sentiments"]=pd.cut(df['Twitter Sentiments'], cut_points, labels=label_names)
   dummies1 = pd.get_dummies(df["sentiments"],drop_first=True)
   df = pd.concat([df, dummies1], axis=1)
   df.drop(["EF_1_val","sentiments"],axis=1,inplace=True)
   return df
def calculate_ema(prices, days, smoothing=2):
      ema = [sum(prices[:days]) / days]
      for price in prices[days:]:
          ema.append((price * (smoothing / (1 + days))) + ema[-1] * (1 - (smoothing / (1 + days))))
      return ema

In [5]:
target=df['FutureTrend']
df=df.drop("FutureTrend",axis =1)
target= target.replace(['Negative','Positive',  'Neutral'],[0,1,2])
df= df.replace(['Negative','Positive',  'Neutral'],[0,1,2])

appl_df =df.set_index('Date',inplace=True)
appl_df = make_bins(df)
appl_df.head(2)
appl_df=appl_df.drop(["Twitter Sentiments"],axis =1)
appl_df['diff']=appl_df['Close']-appl_df['Open']
appl_df['10day']=appl_df['Close'].rolling(10).mean()
appl_df['10day'].fillna(appl_df['10day'][9],inplace=True)
  
ema = calculate_ema(appl_df['Close'], 10)
appl_df=appl_df[9:]
target=target[9:]
appl_df['ema']=ema
appl_df['signal']=appl_df['ema']>appl_df['Close']

del (appl_df['Open'])
del (appl_df['Close'])
del (appl_df['High'])
del (appl_df['EF_2'])

from imblearn.over_sampling import SMOTE
# pd.DataFrame(oe_df)
oversample = SMOTE()
appl_df, target = oversample.fit_resample(appl_df, target)


In [8]:
from sklearn.ensemble import AdaBoostClassifier
new_df=appl_df
print(new_df.head(2))

numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(new_df)
categorical_columns = categorical_columns_selector(new_df)
print(categorical_columns)
preprocessor = ColumnTransformer([
  # ('one-hot-encoder', categorical_preprocessor, categorical_columns),
   ('standard-scaler', numerical_preprocessor, numerical_columns)
    ])
new_df=preprocessor.fit_transform(new_df)
X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2,random_state=42)

                        

         Low   Volume      EF_1  Oil FutureTrend  Gold FutureTrend  Trend  \
0  23.510000  3791500  0.260868                0                 1      0   
1  23.379999  4094700  0.617650                0                 0      1   

   Ef_1_low  Ef_1_v_med  Ef1_high  Ef1_very_high  sentiment_good  \
0         1           0         0              0               0   
1         0           0         1              0               0   

   sentiment_bad      diff   10day        ema  signal  
0              0 -0.120001  23.232  23.232000   False  
1              0  0.210001  23.305  23.297091   False  
[]


In [9]:

clf = RandomForestClassifier(max_depth=2, random_state=5, warm_start=True, n_jobs=-1 )
ada = AdaBoostClassifier(base_estimator=clf, algorithm="SAMME.R", n_estimators=700, learning_rate = .1)
ada.fit(X_train,y_train)

AdaBoostClassifier(base_estimator=RandomForestClassifier(max_depth=2, n_jobs=-1,
                                                         random_state=5,
                                                         warm_start=True),
                   learning_rate=0.1, n_estimators=700)

In [10]:
print("Accuracy score (training): {0:.3f}".format(
        ada.score(
            X_train, 
            y_train)))
print("Accuracy score (validation): {0:.3f}".format(
        ada.score(
            X_test, 
            y_test)))

Accuracy score (training): 0.838
Accuracy score (validation): 0.652


Hyper Parameter tuning 

In [ ]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd

  
seed = 8
kfold = model_selection.KFold(n_splits = 5,
                       )
  
# initialize the base classifier
base_cls = DecisionTreeClassifier()
  
# no. of base classifier
num_trees = 500
for num_trees in [50,100,250,300,400,500]:
# bagging classifier
  model_3 = BaggingClassifier(base_estimator = ada,
                            n_estimators = num_trees,
                            random_state = seed)
  X_train,X_test, y_train, y_test = train_test_split(new_df, target, test_size=0.2, random_state=42)

  model_3.fit(X_train,y_train)
  print("Accuracy score (training): {0:.3f}".format(
        model_3.score(
            X_train, 
            y_train)))
  print("Accuracy score (validation): {0:.3f}".format(
        model_3.score(
            X_test, 
            y_test)))
  # results = model_selection.cross_val_score(model_3, X_train, y_train, cv = kfold)
  # print("accuracy :")
  # print("num_trees :",num_trees,results.mean())

Hyper parameter tuning on Bagging + Ensemble classifier (took too much time)

In [ ]:
# param_grid = {
#     'base_estimator__max_depth' : [1, 2, 3, 4, 5],
#     'max_samples' : [0.05, 0.1, 0.2, 0.5],
#     'n_estimators' : [5,10,20,40,60],
#     'bootstrap' :[True,False]

# }


# #Fitting Bagging Classifier model with  hyper parameters
# # bagg = BaggingClassifier(bootstrap=True,n_estimators=200)
# # gridbag = GridSearchCV(bagg, param_grid=param_grid, cv = 3, verbose = 1)
# # bestbag = gridbag.fit(X_train, y_train)

# gridbag = GridSearchCV(BaggingClassifier(DecisionTreeClassifier(),
#                                      n_estimators = 250, max_features = 0.5),
#                    param_grid)
# gridbag.fit(X_train, y_train)
# print("Accuracy score (training): {0:.3f}".format(
#         gridbag.score(
#             X_train, 
#             y_train)))
# print("Accuracy score (validation): {0:.3f}".format(
#         gridbag.score(
#             X_test, 
#             y_test)))


In [ ]:
# #Printing the best hyperparameters
# print('The best hyper parameters are:\n',gridbag.best_params_)



# #Checking different metrics for bagging model after tuning the hyperparameters
# print('Checking with the best hyper parameter on bagging model after tuning the hyperparameters:\n')

# bestbag=BaggingClassifier(bootstrap= False, max_samples=0.5, n_estimators= 40,max_features=0.6
#                           ,random_state=45
#                           )
# bestbag.fit(X_train,y_train)
# print("Accuracy score (training): {0:.3f}".format(
#         bestbag.score(
#             X_train, 
#             y_train)))
# print("Accuracy score (validation): {0:.3f}".format(
#         bestbag.score(
#             X_test, 
#             y_test)))
